## The Freeman Model

The Freeman model classifies gentrification as occurring an area where the median household income and share of housing built in the prior 20 years are both less than the metro-area values. For an area to be gentrified, the share of residents with college degrees also has to be greater than the metro value, and there needs to be an increase in house prices.

Datasets required:
* Median household income (approximate using yearly median earnings - assume proportionality)
* Housing supply
* Degree equivalent qualifications
* Average house price

In [1]:
import numpy as np
import pandas as pd
import scipy as sp

print("success!")

success!
